In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np
import datetime
from collections import Counter
import time

In [2]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            "00:00:00":'','0:00':'',
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="coerce"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)
            
    def orientation_determine(self,df):
        list_columns = []
        for j in range(df.shape[1]):
            list_columns.append(pd.to_numeric(df.iloc[:,j],errors='coerce').dropna())
        clear_df = pd.concat(list_columns,axis=1)
        if clear_df.shape[0] > clear_df.shape[1]:
            return True
        else: return False
        
    def find_start_point(self,clear_df):
        start_point = 0
        for i in range(clear_df.shape[0]):
            if len(clear_df.iloc[i,:].dropna()) > 2:
                start_point = i
                break
        return start_point
    
    def define_type_of_cell(self,clear_df,df_data,start_point):
        columns_types = {}
        for j in range(clear_df.shape[1]):
            baff = clear_df.iloc[:,j].dropna()
            if baff.sum() != 0:
                if baff.dtype == pd.Float64Dtype():
                    columns_types[j] = 'float'
                else: columns_types[j] = 'int'
            else:
                if type(testTableParser.parse(df_data.iloc[:,j].astype(str))[0][start_point]) == type(pd.to_datetime('31.01.2002')):
                    columns_types[j] = 'date'
                elif baff.dtype == pd.BooleanDtype():
                    columns_types[j] = 'bool'
                else: columns_types[j] = 'str'
        return columns_types
    
    def define_breakline(self,clear_df,columns_types):
        null_list,index_list= [],[]
        for j in range(clear_df.shape[1]):
            if columns_types[j] == 'int':
                null_list.append(clear_df.iloc[:,j])
        int_df = pd.concat(null_list,axis=1)
        count = Counter(columns_types.values())
        for i in range(int_df.shape[0]):
            if int_df.iloc[i,:].isnull().sum() == count.get('int'):
                index_list.append(i)

        return index_list

    def create_list_of_table(self,df,break_line,start_point):
        return [df.iloc[:break_line[0]+start_point,:],df.iloc[break_line[0]+start_point:,:]]
            
    def json_creator(self,df,orientation,columns_types):
        return {"IsVerticalOrientation,":orientation,
               "Index rows":[df.index.tolist()],
               "columns types": columns_types,
                "mistake fix":[{"row": 0,
                                "column": 0,
                                "new value": object,
                                "old value":object,
                                "probability":100,
                                "user approve":False,
                                "comment":bool    
                                }]
               }
        
    def create_header(self,df,start_point):
        topic = df.iloc[:start_point,:].fillna(method='ffill',axis=1)
        list_topic = []
        for i in range(topic.shape[1]):
            list_topic.append("__".join(list(topic.iloc[:,i].astype(str))).replace("__nan","").replace("nan__",""))
        df_data = df.iloc[start_point:]
        df_data.columns = list_topic
        return df_data   

    def to_numeric_func(self,df):
        list_columns = []
        for j in range(df.shape[1]):
            list_columns.append(pd.to_numeric(df.iloc[:,j],errors='coerce').convert_dtypes())
        clear_df = pd.concat(list_columns,axis=1)
#         start = time.time()
#         end = time.time()
#         print(end - start)
        return clear_df

    def all_action(self,df): # <-----    drive method
        orientation = self.orientation_determine(df)
#         orientation = True
        print(orientation)
        if orientation== False:
            df = df.T
            
        clear_df = self.to_numeric_func(df)
        start_point = self.find_start_point(clear_df)
        columns_types = self.define_type_of_cell(clear_df.iloc[start_point:,:],df.iloc[start_point:,:],start_point)
        break_line = self.define_breakline(clear_df.iloc[start_point:,:],columns_types)
        if  len(break_line) == 0:
            json = self.json_creator(df,orientation,columns_types)
#             print("finded innn :"+ str(start_point))
            done_df = self.create_header(df,start_point)
#             print(json)
            return [done_df]

        else:
            list_df = self.create_list_of_table(df,break_line,start_point)
            answer_df = []
            for i in range(len(list_df)):
                clear_df = self.to_numeric_func(list_df[i])
                start_point = self.find_start_point(clear_df)
                done_df = self.create_header(list_df[i],start_point)
                answer_df.append(done_df)
            return answer_df
            


        

In [10]:
# df = pd.read_csv("file44.csv",encoding='unicode_escape',header=None)
testTableParser = TestTableParser()
# df = pd.read_excel(open('file44.xlsx', 'rb'),header=None,sheet_name='Worksheet')
# df = pd.read_excel("file22.xlsx",header=None)
# print(df.shape)
# print(df.iloc[1,:])
# df = testTableParser.all_action(df)
# df[0]

xlsx = pd.ExcelFile('file44.xlsx')
names = xlsx.sheet_names
sheet_to_df_map = {}
for i  in range(len(names)):
    sheet_to_df_map[names[i]] = testTableParser.all_action(pd.read_excel(open('file44.xlsx', 'rb'),header=None,sheet_name=names[i]))
    

True


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname C identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [ ]:
sheet_to_df_map.get('TDSheet')[0]

In [ ]:
# dict_keys(['Стадо 1 ', 'Стадо 2  ', 'Стадо 3 ', 'Стадо 4 ', 'Стадо 5 ', 'Стадо 6', 'Стадо 7', 'Стадо 8', 'Стадо 9', 'Стадо 10 ', 'Стадо 11', 'Стадо 12', 'Стадо 13'])

In [236]:
df = pd.read_excel("file44.xlsx",header=None)
s = pd.Series([234.0,32.9,1,4,14])
# s = pd.to_numeric(df.iloc[:,0],errors='coerce').dropna()
s = s.convert_dtypes()
s.map(type)
# s.dtype == pd.Float64Dtype()
# s.dtype == pd.Int64Dtype()

0    <class 'float'>
1    <class 'float'>
2    <class 'float'>
3    <class 'float'>
4    <class 'float'>
dtype: object

In [8]:
s = pd.Series([234.0,np.nan,32.9,1,4,14,np.nan])
s[s.isnull()].index.tolist()

[1, 6]

In [111]:
s = pd.Series(['a',234.0,32.9,'ayan',1,4,14,'12.4','34.2','21a','a'])
pd.to_numeric(s,errors='coerce').convert_dtypes()

0      <NA>
1     234.0
2      32.9
3      <NA>
4       1.0
5       4.0
6      14.0
7      12.4
8      34.2
9      <NA>
10     <NA>
dtype: Float64

In [112]:
s[s.astype(str).str.contains('a')].index.to_list()

[0, 3, 9, 10]

In [127]:
def find_errors(s):
    mistake = {'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5'}
    finded_error = {}
    for key, val in mistake.items():
        if len(s[s.astype(str).str.contains(key).dropna()].index.to_list()) != 0:
            finded_error[key] = [mistake.get(key),s[s.astype(str).str.contains(key)].index.to_list()]
        s = s.astype(str).str.strip().str.replace(key, val, regex=True)
    return [s,finded_error]

In [128]:
null_list = [pd.Series(['A',234.0,32.9,'E',1,4,14,'l2.4','34D.2','21l','q','S']),
             pd.Series(['22o',234.0,32.9,'b',1,4,14,'12.v','34.g','21F','t'])
            ]
int_df = pd.concat(null_list,axis=1)
# int_df

In [129]:
list_right_data = []
mistake_index_list = []
for j in range(int_df.shape[1]):
#     s = pd.Series(['A.5',234.0,32.9,'E.23',1,4,14,'12.4','34D.2','21','q.2','S.9'])
    buff = find_errors(int_df.iloc[:,j])
    list_right_data.append(pd.to_numeric(buff[0],errors='coerce').convert_dtypes())
    mistake_index_list.append(buff[1])
ans_df = pd.concat(list_right_data,axis=1)

In [130]:
mistake_index_list

[{'l': ['1', [7, 9]],
  'q': ['9', [10]],
  'D': ['0', [8]],
  'E': ['3', [3]],
  'A': ['4', [0]],
  'S': ['5', [11]]},
 {'o': ['0', [0]],
  'b': ['6', [3]],
  'g': ['9', [8]],
  't': ['7', [10]],
  'v': ['5', [7]],
  'F': ['7', [9]]}]

In [131]:
ans_df

,0,1
0,4.0,220.0
1,234.0,234.0
2,32.9,32.9
3,3.0,6.0
4,1.0,1.0
5,4.0,4.0
6,14.0,14.0
7,12.4,12.5
8,340.2,34.9
9,211.0,217.0


In [99]:
ans_df.dtypes

0    Float64
1    Float64
dtype: object

[{'l': [7, 9], 'q': [10], 'D': [8], 'E': [3], 'A': [0], 'S': [11]},
 {'o': [0], 'b': [3], 'g': [8], 't': [10], 'v': [7], 'F': [9]}]

In [119]:
#         print([type(each) for each in df.iloc[:,4]])
# <class 'datetime.datetime'>

In [577]:
df.iloc[:,5].apply(type).value_counts()


<class 'str'>    2824
Name: 5, dtype: int64

In [118]:
 mistake = {'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5'}

In [121]:
mistake.get('l')

'1'